In [1]:
import os
from pyspark.sql import SparkSession

# Set environment variables
os.environ['JAVA_HOME'] = '/home/ymusic7/.conda/envs/myenv'
os.environ['PYSPARK_PYTHON'] = '/home/ymusic7/.conda/envs/myenv/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/ymusic7/.conda/envs/myenv/bin/python'

# Initialize Spark session with correct Python paths
spark = SparkSession.builder \
    .appName("restaurant_binary_analysis") \
    .config("spark.pyspark.python", "/home/ymusic7/.conda/envs/myenv/bin/python") \
    .config("spark.pyspark.driver.python", "/home/ymusic7/.conda/envs/myenv/bin/python") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")  # or "INFO" for more details



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/02 21:07:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import os
print(os.environ['PYSPARK_PYTHON'])
print(os.environ['JAVA_HOME'])


/home/ymusic7/.conda/envs/myenv/bin/python
/home/ymusic7/.conda/envs/myenv


In [3]:
import os
print(os.environ.get('JAVA_HOME'))


/home/ymusic7/.conda/envs/myenv


In [7]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StandardScaler, PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.mllib.feature import StandardScaler as StandardScalerRDD
from pyspark.mllib.linalg.distributed import RowMatrix
import pyspark.sql.functions as F
import csv


# Read Spotify data
df = spark.read.csv('/home/ymusic7/metro_restaurants/combined.csv', header=True)

# Note
df.columns





['name', 'lat', 'lon', 'text', 'date', 'score', 'keyword', 'addr', 'language']

In [6]:
df.head(5)

[Row(name='Koi Sushi', lat='44.6048264', lon='-79.3724546', text="I've been here several times and am never disappointed. Food is served up fresh in record time. On average you can expect a pickup order to be bagged and ready in 20 minutes (30 min during dinner).  Staff are always friendly.", date='2021-01-29 23:56:06', score='5', keyword='japanese restaurant', addr='660 Atherley Rd #1,2, Orillia, ON L3V 1P2, Canada', language='en'),
 Row(name='Koi Sushi', lat='44.6048264', lon='-79.3724546', text="Koi sushi has been great every time I've gone.If you want a good deal go for lunch during Thursday. Or even dinner, we were able to feed three people all you can eat for only $106. The staff is super good and friendly!I always enjoy the shrimp tempura and the salmon sushi.", date='2024-05-27 00:55:48', score='5', keyword='japanese restaurant', addr='660 Atherley Rd #1,2, Orillia, ON L3V 1P2, Canada', language='en'),
 Row(name='Hanabi Japanese Restaurant', lat='44.583675', lon='-79.8655205', 

In [12]:
df.printSchema()



root
 |-- name: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- text: string (nullable = true)
 |-- date: string (nullable = true)
 |-- score: string (nullable = true)
 |-- keyword: string (nullable = true)
 |-- addr: string (nullable = true)
 |-- language: string (nullable = true)



In [8]:
import pyspark.sql.functions as F

# Filter DataFrame where language is English
df_english = df.filter(F.col('language') == 'en')

df_english.show(5)
print(f"Number of reviews in English: {df_english.count()}")


+--------------------+----------+------------------+--------------------+-------------------+-----+-------------------+--------------------+--------+
|                name|       lat|               lon|                text|               date|score|            keyword|                addr|language|
+--------------------+----------+------------------+--------------------+-------------------+-----+-------------------+--------------------+--------+
|           Koi Sushi|44.6048264|       -79.3724546|I've been here se...|2021-01-29 23:56:06|    5|japanese restaurant|660 Atherley Rd #...|      en|
|           Koi Sushi|44.6048264|       -79.3724546|Koi sushi has bee...|2024-05-27 00:55:48|    5|japanese restaurant|660 Atherley Rd #...|      en|
|Hanabi Japanese R...| 44.583675|       -79.8655205|Very cute little ...|2024-09-13 11:21:05|    5|japanese restaurant|5 Queen St E, Elm...|      en|
|           2go sushi|44.6069384|       -80.5923143|Cozy restaurant w...|2024-09-03 12:24:22|    5|j

Number of reviews in English: 480060


In [9]:
import os
from pyspark.sql import SparkSession

# Set environment variables
os.environ['JAVA_HOME'] = '/home/ymusic7/.conda/envs/myenv'
os.environ['PYSPARK_PYTHON'] = '/home/ymusic7/.conda/envs/myenv/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/ymusic7/.conda/envs/myenv/bin/python'

# Start Spark session
spark = SparkSession.builder \
    .appName("restaurant_binary_analysis") \
    .config("spark.pyspark.python", "/home/ymusic7/.conda/envs/myenv/bin/python") \
    .config("spark.pyspark.driver.python", "/home/ymusic7/.conda/envs/myenv/bin/python") \
    .getOrCreate()


In [10]:
import nltk
print(nltk.data.path)


['/home/ymusic7/nltk_data', '/home/ymusic7/.conda/envs/myenv/nltk_data', '/home/ymusic7/.conda/envs/myenv/share/nltk_data', '/home/ymusic7/.conda/envs/myenv/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


In [11]:
import os

# Check the stopwords directory
stopwords_dir = '/home/ymusic7/.conda/envs/myenv/nltk_data/corpora/stopwords'
if os.path.exists(stopwords_dir):
    print("Stopwords directory exists. Contents:")
    print(os.listdir(stopwords_dir))
else:
    print("Stopwords directory does not exist.")


Stopwords directory exists. Contents:
['README', 'spanish', 'catalan', 'norwegian', 'kazakh', 'indonesian', 'arabic', 'greek', 'hebrew', 'slovene', 'russian', 'dutch', 'danish', 'german', 'swedish', 'italian', 'nepali', 'portuguese', 'finnish', 'english', 'hungarian', 'french', 'azerbaijani', 'bengali', 'hinglish', 'basque', 'turkish', 'chinese', 'romanian', 'tajik']


In [12]:
from nltk.corpus import stopwords

# Load the English stopwords
english_stopwords = set(stopwords.words('english'))

# Now you can use 'english_stopwords' in your cleaning process
print("English Stopwords:", english_stopwords)


English Stopwords: {'because', 'hasn', 'he', "doesn't", 'himself', 'what', "hasn't", 'up', 'haven', 'at', 'why', 'where', 'against', 'yours', 'she', 'how', 'in', "she's", 'd', 'should', 'wouldn', 'during', 'whom', "shouldn't", 'themselves', "weren't", "won't", 'o', 'll', 'if', 'other', "you've", 'didn', 'further', 's', "wouldn't", 'they', 'isn', 'an', 'having', 'is', 'such', 'so', 're', 'no', 'your', 'm', 'mightn', 'be', 'now', "haven't", "you're", 'under', 'does', 'below', 'couldn', 'was', 'been', 'until', "should've", 'ma', 'before', 'nor', 'with', 've', 'these', "you'd", 'them', 'by', 'off', 'am', 'you', 'i', 'to', "shan't", 'wasn', 'do', 'a', 'our', 'here', 'about', 'then', 'ourselves', 'don', 'when', 'are', 't', 'most', 'through', "you'll", "hadn't", 'only', "that'll", 'doing', 'her', 'ours', 'own', "mustn't", 'theirs', "needn't", 'over', 'not', "didn't", 'has', 'on', 'all', 'while', 'same', 'just', 'as', 'yourselves', "couldn't", "mightn't", 'weren', 'my', 'and', 'there', 'me', "

In [13]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import Tokenizer
import pyspark.sql.functions as F
import re
import nltk
from nltk.corpus import stopwords

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("RestaurantReviewAnalysis") \
    .getOrCreate()

# Load your initial DataFrame (replace 'your_dataframe' with your actual DataFrame)
# df = spark.read.csv("path_to_your_data.csv", header=True, inferSchema=True)

# Assuming 'text' is the column with the reviews and 'keyword' contains cuisine information
# Create a tokenized DataFrame
tokenizer = Tokenizer(inputCol="text", outputCol="words")
df_tokenized = tokenizer.transform(df)

# Load the English stopwords

english_stopwords = set(stopwords.words('english'))

# Additional stopwords
additional_stopwords = english_stopwords.union({
    "one", "two", "three", "four", "five", "six", "seven", 
    "eight", "nine", "ten", "to", "so", "thats", "then", 
    "for", "up", "its", "well", "with", "you", "your", 
    "i", "we", "me", "my"
})

# Function to clean words list
def advanced_clean_words(words_list):
    if words_list is None:
        return []
    cleaned = [
        re.sub(r'[^\w\s]', '', word).lower()
        for word in words_list
        if word.lower() not in additional_stopwords and not word.isdigit()
    ]
    seen = set()
    unique_words = []
    for word in cleaned:
        if word not in seen:
            seen.add(word)
            unique_words.append(word)
    return unique_words

# Register the UDF
clean_words_udf = udf(lambda words: advanced_clean_words(words), ArrayType(StringType()))

# Drop rows with nulls in 'words' column before applying the UDF
df_tokenized = df_tokenized.na.drop(subset=["words"])  # Drop rows with nulls in 'words'

# Apply the UDF to clean the text in the 'words' column and change output column name
df_cleaned = df_tokenized.withColumn('filtered_text', clean_words_udf(F.col('words')))

# Filter for Japanese and Chinese restaurant reviews
df_japanese = df_cleaned.filter(F.col('keyword').contains('japanese restaurant'))
df_chinese = df_cleaned.filter(F.col('keyword').contains('chinese restaurant'))

# Further divide based on sentiment score
df_japanese_positive = df_japanese.filter(F.col('score') > 3).select('name', 'filtered_text', 'score', 'keyword')
df_japanese_negative = df_japanese.filter(F.col('score') < 3).select('name', 'filtered_text', 'score', 'keyword')
df_chinese_positive = df_chinese.filter(F.col('score') > 3).select('name', 'filtered_text', 'score', 'keyword')
df_chinese_negative = df_chinese.filter(F.col('score') < 3).select('name', 'filtered_text', 'score', 'keyword')

# Show the first few rows of each DataFrame for verification
print("Japanese Positive Reviews:")
df_japanese_positive.show(5, truncate=False)

print("Japanese Negative Reviews:")
df_japanese_negative.show(5, truncate=False)

print("Chinese Positive Reviews:")
df_chinese_positive.show(5, truncate=False)

print("Chinese Negative Reviews:")
df_chinese_negative.show(5, truncate=False)


Japanese Positive Reviews:


+--------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-------------------+
|name                      |filtered_text                                                                                                                                                                                                                                                                                |score|keyword            |
+--------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.sql import functions as F


In [12]:
df_cleaned.printSchema()


root
 |-- name: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- text: string (nullable = true)
 |-- date: string (nullable = true)
 |-- score: string (nullable = true)
 |-- keyword: string (nullable = true)
 |-- addr: string (nullable = true)
 |-- language: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered_text: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [13]:
from pyspark.sql import functions as F

# Remove null values from the 'filtered_text' array
df_cleaned = df_cleaned.withColumn(
    "filtered_text", 
    F.expr("filter(filtered_text, x -> x is not null)")
)
df_cleaned = df_cleaned.filter(F.size(F.col("filtered_text")) > 0)
df_cleaned.printSchema()




root
 |-- name: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- text: string (nullable = true)
 |-- date: string (nullable = true)
 |-- score: string (nullable = true)
 |-- keyword: string (nullable = true)
 |-- addr: string (nullable = true)
 |-- language: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered_text: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [14]:
from pyspark.ml.feature import HashingTF, IDF

# Function to apply TF-IDF
def apply_tfidf(df, feature_col):
    # Create HashingTF
    hashing_tf = HashingTF(inputCol=feature_col, outputCol="rawFeatures", numFeatures=10000) #10000 for each 4 categories
    featurized_data = hashing_tf.transform(df)
    
    # Create IDF
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    idfModel = idf.fit(featurized_data)
    tfidf_data = idfModel.transform(featurized_data)
    
    return tfidf_data

# Apply TF-IDF to each filtered DataFrame
tfidf_japanese_positive = apply_tfidf(df_japanese_positive, "filtered_text")
tfidf_japanese_negative = apply_tfidf(df_japanese_negative, "filtered_text")
tfidf_chinese_positive = apply_tfidf(df_chinese_positive, "filtered_text")
tfidf_chinese_negative = apply_tfidf(df_chinese_negative, "filtered_text")


# Verify the results
tfidf_japanese_positive.select("name", "features").show(5, truncate=False)
tfidf_japanese_negative.select("name", "features").show(5, truncate=False)
tfidf_chinese_positive.select("name", "features").show(5, truncate=False)
tfidf_chinese_negative.select("name", "features").show(5, truncate=False)



+--------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|name                      |featur

In [16]:
# Save TF-IDF results to CSV files
# Save TF-IDF results to Parquet files
tfidf_japanese_positive.write.parquet("/home/ymusic7/metro_restaurants/tfidf_japanese_positive.parquet")
tfidf_japanese_negative.write.parquet("/home/ymusic7/metro_restaurants/tfidf_japanese_negative.parquet")
tfidf_chinese_positive.write.parquet("/home/ymusic7/metro_restaurants/tfidf_chinese_positive.parquet")
tfidf_chinese_negative.write.parquet("/home/ymusic7/metro_restaurants/tfidf_chinese_negative.parquet")



In [17]:
from pyspark.sql.functions import split
# Remove rows where 'filtered_text' is null
df_cleaned = df_cleaned.na.drop(subset=["filtered_text"])


# Convert 'filtered_text' back to an array
df_cleaned = df_cleaned.withColumn('filtered_text', split(F.col('filtered_text'), ' '))

# Apply TF-IDF to the cleaned DataFrame
tfidf_all_reviews = apply_tfidf(df_cleaned, "filtered_text")

# Verify the results
tfidf_all_reviews.select("name", "features").show(5, truncate=False)



AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "split(filtered_text,  , -1)" due to data type mismatch: Parameter 1 requires the "STRING" type, however "filtered_text" has the type "ARRAY<STRING>".;
'Project [name#52, lat#53, lon#54, text#55, date#56, score#57, keyword#58, addr#59, language#60, words#135, split(filtered_text#160,  , -1) AS filtered_text#510]
+- Filter atleastnnonnulls(1, filtered_text#160)
   +- Project [name#52, lat#53, lon#54, text#55, date#56, score#57, keyword#58, addr#59, language#60, words#135, <lambda>(words#135)#159 AS filtered_text#160]
      +- Filter atleastnnonnulls(1, words#135)
         +- Project [name#52, lat#53, lon#54, text#55, date#56, score#57, keyword#58, addr#59, language#60, UDF(text#55) AS words#135]
            +- Relation [name#52,lat#53,lon#54,text#55,date#56,score#57,keyword#58,addr#59,language#60] csv


In [16]:
from pyspark.ml.feature import PCA

# Function to apply PCA
def apply_pca(df, num_components=2):
    pca = PCA(k=num_components, inputCol="features", outputCol="pcaFeatures")
    model = pca.fit(df)
    pca_result = model.transform(df)
    return pca_result

# Applying PCA to TF-IDF results
pca_japanese_positive = apply_pca(tfidf_japanese_positive)
pca_japanese_negative = apply_pca(tfidf_japanese_negative)
pca_chinese_positive = apply_pca(tfidf_chinese_positive)
pca_chinese_negative = apply_pca(tfidf_chinese_negative)

# Show PCA results
pca_japanese_positive.select("name", "pcaFeatures").show(5, truncate=False)
pca_japanese_negative.select("name", "pcaFeatures").show(5, truncate=False)
pca_chinese_positive.select("name", "pcaFeatures").show(5, truncate=False)
pca_chinese_negative.select("name", "pcaFeatures").show(5, truncate=False)


24/11/02 21:02:06 ERROR Executor: Exception in task 5.0 in stage 28.0 (TID 84)8]
java.lang.OutOfMemoryError: Java heap space
24/11/02 21:02:06 ERROR Executor: Exception in task 7.0 in stage 28.0 (TID 86)
java.lang.OutOfMemoryError: Java heap space
	at scala.reflect.ManifestFactory$DoubleManifest.newArray(Manifest.scala:194)
	at scala.reflect.ManifestFactory$DoubleManifest.newArray(Manifest.scala:191)
	at breeze.linalg.DenseVector$mcD$sp.<init>(DenseVector.scala:67)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix.$anonfun$computeGramianMatrix$1(RowMatrix.scala:143)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix$$Lambda$4202/0x0000000101949040.apply(Unknown Source)
	at scala.collection.TraversableOnce$folder$1.apply(TraversableOnce.scala:196)
	at scala.collection.TraversableOnce$folder$1.apply(TraversableOnce.scala:194)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterato

ConnectionRefusedError: [Errno 111] Connection refused

Traceback (most recent call last):
  File "/home/ymusic7/.conda/envs/myenv/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/ymusic7/.conda/envs/myenv/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/ymusic7/.conda/envs/myenv/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/ymusic7/.conda/envs/myenv/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/ymusic7/.conda/envs/myenv/lib/python3.8/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/home/ymusic7/.conda/envs/myenv/lib/python3.8/site-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/home/ymusic7/.conda/envs/myenv/lib/python3.8/site-packages/pyspark/accumulators.py", line 

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA

# Function to apply PCA
def apply_pca(df, num_components=2):
    pca = PCA(k=num_components, inputCol="features", outputCol="pcaFeatures")
    model = pca.fit(df)
    pca_result = model.transform(df)
    return pca_result

if __name__ == "__main__":
    # Initialize Spark session
    spark = SparkSession.builder \
        .appName("restaurant_analysis") \
        .config("spark.driver.memory", "8g") \
        .config("spark.executor.memory", "8g") \
        .getOrCreate()

    # Load TF-IDF DataFrames
    tfidf_japanese_positive = spark.read.parquet("/home/ymusic7/metro_restaurants/tfidf_japanese_positive.parquet")
    tfidf_japanese_negative = spark.read.parquet("/home/ymusic7/metro_restaurants/tfidf_japanese_negative.parquet")
    tfidf_chinese_positive = spark.read.parquet("/home/ymusic7/metro_restaurants/tfidf_chinese_positive.parquet")
    tfidf_chinese_negative = spark.read.parquet("/home/ymusic7/metro_restaurants/tfidf_chinese_negative.parquet")

    # Apply PCA to each DataFrame one by one
    try:
        pca_japanese_positive = apply_pca(tfidf_japanese_positive)
        pca_japanese_positive.select("name", "pcaFeatures").show(5, truncate=False)
        
        #pca_japanese_negative = apply_pca(tfidf_japanese_negative)
        #pca_japanese_negative.select("name", "pcaFeatures").show(5, truncate=False)

        #pca_chinese_positive = apply_pca(tfidf_chinese_positive)
        #pca_chinese_positive.select("name", "pcaFeatures").show(5, truncate=False)

        #pca_chinese_negative = apply_pca(tfidf_chinese_negative)
        #pca_chinese_negative.select("name", "pcaFeatures").show(5, truncate=False)

    except Exception as e:
        print(f"An error occurred: {e}")


An error occurred: An error occurred while calling o79.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 3 tasks (1150.1 MiB) is bigger than spark.driver.maxResultSize (1024.0 MiB)
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1247)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSet

24/11/03 18:16:08 ERROR TaskSetManager: Total size of serialized results of 3 tasks (1150.1 MiB) is bigger than spark.driver.maxResultSize (1024.0 MiB)
24/11/03 18:16:08 WARN TaskSetManager: Lost task 1.0 in stage 25.0 (TID 69) (midway2-0171.rcc.local executor driver): TaskKilled (Tasks result size has exceeded maxResultSize)
24/11/03 18:16:08 ERROR TaskSetManager: Total size of serialized results of 4 tasks (1533.5 MiB) is bigger than spark.driver.maxResultSize (1024.0 MiB)
24/11/03 18:16:08 WARN TaskSetManager: Lost task 3.0 in stage 25.0 (TID 71) (midway2-0171.rcc.local executor driver): TaskKilled (Tasks result size has exceeded maxResultSize)


In [ ]:
#PCA loses interpretability: the reduced features are not easily linked to specific words or topics?

In [ ]:
#Stop Spark session after completing all operations
spark.stop()